<a href="https://colab.research.google.com/github/Gopinath-77/sentimental-analysis---chatbot-/blob/main/huggingface_hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate huggingface_hub

from huggingface_hub import login
login()

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.use_default_system_prompt = False

conversation = []

print("Chatbot initialized. You can start chatting (type 'exit' to stop).")

while True:
    user_input = input("User: ")
    if user_input.lower() == "exit":
        print("Exiting chat.")
        break

    conversation.append({"role": "user", "content": user_input})

    inputs = tokenizer.apply_chat_template(
        conversation,
        return_tensors="pt",
        add_special_tokens=False,
        add_generation_prompt=True
    )
    inputs = inputs.to(model.device) if hasattr(inputs, "to") else {k: v.to(model.device) for k, v in inputs.items()}

    outputs = model.generate(**inputs, max_new_tokens=150)

    input_len = inputs["input_ids"].shape[1]
    response = tokenizer.decode(outputs[0][input_len:], skip_special_tokens=True)

    print(f"Chatbot: {response}")

    conversation.append({"role": "assistant", "content": response})

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

Chatbot initialized. You can start chatting (type 'exit' to stop).
